# Сравнение дообучения Transformers и Adapters

## Цель
Сравнить производительность и эффективность дообучения модели BERT и DistilBERT полное и с адаптерами на задаче классификации новостей (AG News).

## Методология
- **Датасет**: AG News (4 класса: World, Sports, Business, Sci/Tech)
- **Модели**:
  - BERT-base-uncased
  - DistilBERT-base-uncased
  - Адаптеры SeqBn
- **Метрики**: Accuracy, Precision, Recall, F1 (macro-averaged)
- **Гиперпараметры**:
  - Эпохи: 8
  - Размер батча: 32
  - Скорость обучения: 2e-5

## Environment loading

Устанавливаются следующие библиотеки:

* transformers – библиотека от Hugging Face для работы с NLP-моделями (BERT, GPT и др.).

* evaluate – инструменты для оценки моделей машинного обучения.

* datasets – датасеты для NLP от Hugging Face.

* wandb – Weights & Biases, инструмент для трекинга ML-экспериментов.

* adapters – адаптеры для тонкой настройки моделей (например, AdapterHub).

In [22]:
!pip install -q transformers==4.48.3 evaluate==0.4.3 datasets==3.6.0 wandb==0.19.10 adapters==1.1.1 # -q для тихой установки

In [23]:
# Стандартные библиотеки
import os
import gc
import time
import random
from datetime import datetime
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# Сторонние библиотеки
import torch
import wandb
import evaluate
import datasets
from sklearn.model_selection import train_test_split

# Transformers и Adapters
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from adapters import (
    AutoAdapterModel,
    SeqBnConfig,
    init,
)

# Оптимизация инициализации WANDB
use_wandb = True
if use_wandb:
    os.environ["WANDB_API_KEY"] = "b23d2ea7b9cd7899f894c389ecd999f17367498c"
    os.environ["WANDB_PROJECT"] = "SentAnalysis_BERT_vs_Adapter_DistilBERT"
    os.environ["WANDB_LOG_MODEL"] = "end"
    os.environ["WANDB_WATCH"] = "gradients"
    wandb.login()
else:
    os.environ["WANDB_MODE"] = "disabled"

In [24]:
# проверим доступность GPU (CUDA) и настроим PyTorch для оптимальной работы
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # ускоряет свёртки на GPU
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: Tesla T4


In [25]:
# Установка seed для воспроизводимости результатов
random.seed(42)        # Для встроенного модуля random (Python)
np.random.seed(42)     # Для NumPy
torch.manual_seed(42)  # Для PyTorch (CPU и CUDA)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)  # Для воспроизводимости на GPU

# Гиперпараметры обучения
num_train_epochs = 8    # Количество эпох обучения
batch_size = 32         # Размер батча
learning_rate = 2e-5    # Скорость обучения

## Loading the dataset

Загрузим датасет AG News (новостной классификационный датасет)

**Источник**: Собран из новостных статей 4-х категорий:
* World - 0;
* Sports - 1;
* Business - 2;
* Sci/Tech - 3.

**Размер**: 120K примеров (30K на класс).

**Структура**:

* train (обучающая выборка) – 120,000 примеров;

* test (тестовая выборка) – 7,600 примеров.

https://huggingface.co/datasets/fancyzhx/ag_news

In [26]:
dataset = datasets.load_dataset("ag_news")
dataset["train"][100] #выведем 101-ый элемент датасета

{'text': 'Comets, Asteroids and Planets around a Nearby Star (SPACE.com) SPACE.com - A nearby star thought to harbor comets and asteroids now appears to be home to planets, too. The presumed worlds are smaller than Jupiter and could be as tiny as Pluto, new observations suggest.',
 'label': 3}

In [27]:
def show_random_elements(dataset, num_examples=10):
    """Отображает случайные примеры из датасета в виде форматированной HTML-таблицы.

    Преобразует числовые метки классов в текстовые названия (если возможно)
    и выводит результат в Jupyter Notebook как интерактивную таблицу.
    Args:
        dataset (datasets.Dataset): Датасет из библиотеки HuggingFace datasets
        num_examples (int): Количество случайных примеров для отображения (по умолчанию 10)
    Returns:
        None: Результат выводится как HTML-таблица в Jupyter Notebook
    """
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), num_examples)
    df = pd.DataFrame(dataset[picks])

    # Преобразование числовых меток в текстовые названия классов
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    # Красивый вывод в Jupyter Notebook
    display(HTML(df.to_html(classes="table table-striped table-bordered")))

In [28]:
# выведем 10 случайных элементов датасета
show_random_elements(dataset["train"], 10)

,text,label
0,"Policeman 'saw fatal train crash' An off-duty policeman watched a train plough into a car on a level crossing in Berkshire, killing six people.",World
1,"Silver finale for USA In the last event of the 2004 Olympic Games, the United States track team produced one last surprise. Meb Keflezighi, a native of Eritrea who moved to the United States as",Sports
2,"Compuware Blasts IBM #39;s Legal Tactics Two years ago, IBM was ordered to produce the source code for its products, which Compuware identified as containing its pirated intellectual property. The code was missing. But lo and behold -- last week, they called and said they had it, quot; ...",Sci/Tech
3,"Polish Hostage Freed in Iraq Already in Warsaw WARSAW (Reuters) - A Polish woman kidnapped in Iraq last month has been freed and flown to Poland and said she was treated well, raising hopes for other foreign hostages.",World
4,"Growth forecast revised up to 7.5pc The Asian Development Bank has revised up its economic growth forecast for Hong Kong this year to 7.5 per cent from the 6 per cent it projected in April, due to stronger than expected retail sales and the surge in tourist arrivals.",Business
5,"File and Printer Sharing Insecure in XP SP2 ProKras writes quot;German magazine PC-Welt has discovered a major security flaw in Windows XP SP2 when installing over SP1. The article says that #39;with a certain configuration, your file and printer sharing data",Sci/Tech
6,"Wall Street Journal to Start Saturday Issue he Wall Street Journal announced yesterday that it would begin publishing a Saturday issue starting in the fall of 2005. By following its business readers home for the weekend, the newspaper",Business
7,"EU, Japan win WTO approval to impose duties on US The European Union, Japan, Brazil and five other countries won World Trade Organization approval to impose tariffs worth more than \$150 million a year on imports from the United",Business
8,Greenspan Warns That US Deficits Pose Risk to Dollar Alan Greenspan came to the home of the euro today and warned anxious Europeans to expect little relief from the relentless decline of the dollar against their currency.,Business
9,"Rivers Hot, Then Cold San Diego Chargers quarterback began the night with three straight completions. Unfortunately, after seven more attempts, he still had only three, plus two interceptions vs. Seattle on Friday.",Sports


## Preprocessing the data

Прежде чем мы сможем передать эти тексты нашей модели, нам нужно их предварительно обработать. Это делается с помощью Transformers Tokenizer, который (как следует из названия) токенизирует входные данные (включая преобразование токенов в соответствующие им идентификаторы в предварительно обученном словаре) и преобразует их в формат, ожидаемый моделью, а также генерирует другие входные данные, требуемые моделью.

Чтобы сделать все это, мы создаем экземпляр нашего токенизатора с помощью метода AutoTokenizer.from_pretrained, который гарантирует:

- мы получаем токенизатор, соответствующий архитектуре модели, которую мы хотим использовать,
- мы загружаем словарь, используемый при предварительном обучении этой конкретной контрольной точки.

In [29]:
# Загрузка предобученного токенизатора BERT (версия base, без учета регистра)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [30]:
# Токенизация пары предложений
encoded_input = tokenizer(
    "Hello, this one sentence!",  # Первое предложение
    "And this sentence goes with it.",  # Второе предложение
    padding=True,  # Дополнение до максимальной длины
    truncation=True,  # Обрезка если текст длиннее максимального размера
    return_tensors="pt"  # Возврат тензоров PyTorch
)
encoded_input
# Результат содержит:
# input_ids - индексы токенов
# token_type_ids - идентификаторы предложений (0/1)
# attention_mask - маска внимания (1 - реальные токены, 0 - паддинг)

{'input_ids': tensor([[ 101, 7592, 1010, 2023, 2028, 6251,  999,  102, 1998, 2023, 6251, 3632,
         2007, 2009, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Для предварительной обработки нашего набора данных нам понадобятся имена столбцов, содержащих предложение(я). Следующий словарь отслеживает задачу соответствия именам столбцов:

In [31]:
def tokenize_function(examples):
    """Применяет токенизатор к пакету примеров текста.

    Args:
        examples (dict): Пакет примеров из датасета, содержащий ключ "text"

    Returns:
        dict: Словарь с токенизированными результатами:
            - input_ids: списки идентификаторов токенов
            - attention_mask: маски внимания (1 для реальных токенов)
            (без padding для эффективной пакетной обработки)
    """
    return tokenizer(examples["text"], truncation=True)  # Без padding!
tokenized_datasets = dataset.map(tokenize_function,
    batched=True  # Обработка пакетами для ускорения
)

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Библиотека Datasets автоматически кэширует результаты предобработки, что позволяет избежать повторных вычислений при следующем запуске блокнота. При этом Datasets достаточно интеллектуальна, чтобы определить, была ли изменена функция, переданная в map — в таком случае кэш не используется. Например, если вы измените задачу в первой ячейке и перезапустите ноутбук, библиотека корректно распознает это и обновит данные.

Если вы хотите отключить использование кэша (например, для принудительного повторения предобработки), передайте аргумент load_from_cache_file=False в метод map. В этом случае Datasets предупредит вас о том, что кэшированные данные не загружаются.

Пакетная обработка
Мы также указали batched=True, чтобы токенизировать тексты не по одному, а целыми партиями. Это особенно важно для эффективного использования быстрого токенизатора (например, Rust-реализации в Hugging Face), который задействует многопоточность для параллельной обработки текстов в пакете.

In [32]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [33]:
#Для уменьшения занимаемой оперативной памяти, удалим исходный текст
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

Выполним разделение датасета на обучающую, валидационную и тестовую выборки, включая создание стратифицированных подмножеств.

In [34]:
train_val_split = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=42)
full_train_dataset = train_val_split["train"]   # 108,000 примеров (90%)
full_eval_dataset = train_val_split["test"]     # 12,000 примеров (10%)
full_test_dataset = tokenized_datasets["test"]  # 7,600 (исходный тестовый набор)

def stratified_subset(dataset, size, seed=42):
    """Создает стратифицированную подвыборку, сохраняя распределение классов."""
    labels = dataset["label"]
    indices = np.arange(len(dataset))
    _, subset_indices = train_test_split(
        indices, test_size=size, stratify=labels, random_state=seed
    )
    return dataset.select(subset_indices)

small_train_dataset = stratified_subset(tokenized_datasets["train"], size=21600, seed=42)  # ~20% от train
small_eval_dataset = stratified_subset(tokenized_datasets["train"], size=9600, seed=42)    # ~8% от train
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(3800))       # 50% от test

train_dataset, eval_dataset, test_dataset = small_train_dataset, small_eval_dataset, small_test_dataset
# Альтернатива: full_train_dataset, full_eval_dataset, full_test_dataset

## Load metrics

Для объективной оценки модели мы будем использовать стандартные метрики из библиотеки evaluate. Этот пакет предоставляет:

* Предопределённые метрики (точность, F1, recall и другие) с оптимизированными
реализациями.

* Стандартизированный интерфейс для вычислений, совместимый с 🤗 Transformers.

* Воспроизводимость — гарантия одинаковых результатов при одинаковых входных данных.

In [35]:
# Загрузка метрик классификации
accuracy_metric = evaluate.load("accuracy")    # Точность (доля правильных ответов)
recall_metric = evaluate.load("recall")       # Полнота (способность находить все положительные случаи)
precision_metric = evaluate.load("precision") # Точность (доля релевантных результатов среди предсказанных)
f1_metric = evaluate.load("f1")               # F1-score (гармоническое среднее precision и recall)

In [36]:
def compute_metrics(eval_pred):
    """Вычисляет метрики качества классификации для оценки модели.

    Args:
        eval_pred (tuple): Кортеж, содержащий:
            - logits (np.ndarray): Предсказания модели (до преобразования в вероятности)
            - labels (np.ndarray): Истинные метки классов

    Returns:
        dict: Словарь с вычисленными метриками:
            - accuracy (точность)
            - recall (полнота)
            - precision (точность для классов)
            - f1 (F1-мера)

    Примечания:
        - Использует макро-усреднение для многоклассовой классификации (все классы равнозначны)
        - Подходит как для бинарной, так и для многоклассовой классификации
        - Совместима с Hugging Face Trainer
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)['accuracy']
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro')['recall']
    precision = precision_metric.compute(predictions=predictions, references=labels, average='macro')['precision']
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1']

    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

## Transformers model

### BERT

Загрузим предварительно обученную модель и настроим ее. Поскольку все наши задачи связаны с классификацией предложений, мы используем класс AutoModelForSequenceClassification. Как и в случае с токенизатором, метод from_pretrained загрузит и кэширует для нас модель. Единственное, что нам нужно указать, — это количество меток для нашей задачи.
https://huggingface.co/google-bert/bert-base-uncased

In [37]:
# Загрузка предобученной модели BERT для классификации текста
model_bert = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Название модели: BERT базовой версии (без учета регистра)
    num_labels=4,         # Количество классов для классификации
    ignore_mismatched_sizes=False  # Игнорировать несоответствия размеров выходного слоя
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Английская версия BERT (12 слоёв, 768 скрытых нейронов), не учитывает регистр (текст преобразуется в нижний регистр).

Настройка:

* num_labels=4 — классификация на 4 класса.

* Автоматически добавляется классификационный слой.

Внутренняя работа:

* Загружаются предобученные веса BERT.

* Последний слой заменяется на классификатор с 4 выходами.

* Веса нового слоя инициализируются случайно.

Для создания экземпляра Trainer нам нужно определить, помимо метрик, еще одну вещь. Наиболее важным является TrainingArguments, который является классом, содержащим все атрибуты для настройки обучения. Он требует одно имя папки, которое будет использоваться для сохранения контрольных точек модели, а все остальные аргументы являются необязательными

In [38]:
training_args = TrainingArguments(
    output_dir="./bert_agnews_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch", #современный оптимизатор для трансформеров
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1", #выбор модели по F1-score
    greater_is_better=True,
    report_to="wandb", # визуализация в Weights & Biases
    logging_steps=50, #частота вывода логов
    save_total_limit=2, #предотвращает переполнение диска
    fp16=True,  # Автоматически использует mixed precision если доступно
    gradient_accumulation_steps=1, #полезно для больших батчей
    warmup_ratio=0.1, #плавное увеличение learning rate в начале
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
# Инициализация тренера для обучения модели
trainer_bert = Trainer(
    # Основные компоненты
    model=model_bert,  # Загруженная ранее модель BERT для классификации
    args=training_args,  # Параметры обучения, которые мы настроили ранее

    # Данные для обучения и оценки
    train_dataset=train_dataset,  # Обучающий датасет
    eval_dataset=eval_dataset,    # Валидационный датасет

    # Настройки обработки данных
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,  # Токенизатор для динамического добавления padding
        padding='longest',    # Дополнение до длины самого длинного примера в батче
        max_length=512,       # Максимальная длина последовательности (ограничение BERT)
        return_tensors='pt'   # Возвращать тензоры PyTorch
    ),

    # Метрики оценки
    compute_metrics=compute_metrics,  # Функция для расчета метрик качества

    # Коллбэки
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Остановить обучение, если метрика не улучшается 3 эпохи
            early_stopping_threshold=0.01  # Минимальное улучшение для продолжения обучения
        )
    ]
)

In [40]:
# Инициализация мониторинга в Weights & Biases
wandb.init(
    project="SentAnalysis_BERT_vs_Adapter_DistilBERT",  # Название проекта в W&B
    config=training_args.to_dict(),  # Сохранение всех параметров обучения
    name=f"BERT-base-{datetime.now().strftime('%Y-%m-%d_%H-%M')}",  # Уникальное имя запуска
    tags=["baseline", "bert-base"]  # Теги для фильтрации экспериментов
)

try:
    # Основной цикл обучения
    trainer_bert.train()  # Запуск обучения с ранней остановкой
    trainer_bert.evaluate()
finally:
    # Обязательное завершение сессии W&B
    wandb.finish()

    # Оптимизация использования памяти GPU
    torch.cuda.empty_cache()  # Очистка кэша CUDA
    gc.collect()             # Запуск сборщика мусора

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.258800,0.181073,0.940104,0.940104,0.940065,0.939986
2,0.162900,0.098053,0.970000,0.970000,0.970867,0.969988
3,0.105700,0.053934,0.985000,0.985000,0.985114,0.985004
4,0.071800,0.023712,0.993229,0.993229,0.993245,0.993232
5,0.056500,0.009202,0.997708,0.997708,0.997710,0.997708
6,0.023400,0.005069,0.998750,0.998750,0.998752,0.998750


eval/accuracy,▁▅▆▇███
eval/f1,▁▅▆▇███
eval/loss,█▅▃▂▁▁▁
eval/precision,▁▅▆▇███
eval/recall,▁▅▆▇███
eval/runtime,▁▂▄▃█▆▅
eval/samples_per_second,█▇▅▆▁▃▄
eval/steps_per_second,█▇▅▆▁▃▄
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train/grad_norm,▃▂▃▃▃▃▂▂▃▂▅▄▃▃▃▁▃▂▃▂▂▃▁▂▂▅▁▁▂▁▁▁▁▁▁▃▁▁▁█


### DistilBERT

Аналогично проведем дообучение модели DistilBERT. Чтобы исключить влияние токенизатора, будем использовать один для всех экспериментов (BERT tokenizator).
* Обе модели используют одинаковый словарь (30,522 токенов)
* Одинаковые специальные токены ([CLS], [SEP])
* Совместимые методы токенизации (WordPiece)
* В обоих случаях используются uncased-версий моделей

https://huggingface.co/distilbert/distilbert-base-uncased

In [42]:
model_distilbert = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir="./bert_agnews_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch", #современный оптимизатор для трансформеров
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1", #выбор модели по F1-score
    greater_is_better=True,
    report_to="wandb", # визуализация в Weights & Biases
    logging_steps=50, #частота вывода логов
    save_total_limit=2, #предотвращает переполнение диска
    fp16=True,  # Автоматически использует mixed precision если доступно
    gradient_accumulation_steps=1, #полезно для больших батчей
    warmup_ratio=0.1, #плавное увеличение learning rate в начале
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:
# Инициализация тренера для обучения модели
trainer_distilbert = Trainer(
    # Основные компоненты
    model= model_distilbert,  # Загруженная ранее модель DistilBERT для классификации
    args=training_args,  # Параметры обучения, которые мы настроили ранее

    # Данные для обучения и оценки
    train_dataset=train_dataset,  # Обучающий датасет
    eval_dataset=eval_dataset,    # Валидационный датасет
    # Настройки обработки данных
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,  # Токенизатор для динамического добавления padding
        padding='longest',    # Дополнение до длины самого длинного примера в батче
        max_length=512,       # Максимальная длина последовательности (ограничение BERT)
        return_tensors='pt'   # Возвращать тензоры PyTorch
    ),

    # Метрики оценки
    compute_metrics=compute_metrics,  # Функция для расчета метрик качества

    # Коллбэки
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Остановить обучение, если метрика не улучшается 3 эпохи
            early_stopping_threshold=0.01  # Минимальное улучшение для продолжения обучения
        )
    ]
)


In [46]:
# Инициализация мониторинга в Weights & Biases
wandb.init(
    project="SentAnalysis_BERT_vs_Adapter_DistilBERT",  # Название проекта в W&B
    config=training_args.to_dict(),  # Сохранение всех параметров обучения
    name=f"DistilBERT-base-{datetime.now().strftime('%Y-%m-%d_%H-%M')}",  # Уникальное имя запуска
    tags=["baseline", "bert-base"]  # Теги для фильтрации экспериментов
)

try:
    # Основной цикл обучения
  trainer_distilbert.train()  # Запуск обучения с ранней остановкой
  trainer_distilbert.evaluate()
finally:
    # Обязательное завершение сессии W&B
    wandb.finish()

    # Оптимизация использования памяти GPU
    torch.cuda.empty_cache()  # Очистка кэша CUDA
    gc.collect()             # Запуск сборщика мусора

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.299600,0.207760,0.934271,0.934271,0.934400,0.934157
2,0.177600,0.119801,0.962083,0.962083,0.963321,0.962013
3,0.125200,0.069070,0.980104,0.980104,0.980127,0.980105
4,0.091900,0.037900,0.990417,0.990417,0.990494,0.990412
5,0.073100,0.022465,0.994896,0.994896,0.994895,0.994895


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.299600,0.207760,0.934271,0.934271,0.934400,0.934157
2,0.177600,0.119801,0.962083,0.962083,0.963321,0.962013
3,0.125200,0.069070,0.980104,0.980104,0.980127,0.980105
4,0.091900,0.037900,0.990417,0.990417,0.990494,0.990412
5,0.073100,0.022465,0.994896,0.994896,0.994895,0.994895
6,0.035600,0.014460,0.996875,0.996875,0.996882,0.996874
7,0.027700,0.008850,0.998229,0.998229,0.998229,0.998229


eval/accuracy,▁▄▆▇████
eval/f1,▁▄▆▇████
eval/loss,█▅▃▂▁▁▁▁
eval/precision,▁▄▆▇████
eval/recall,▁▄▆▇████
eval/runtime,█▁▃▃▂▂▂▄
eval/samples_per_second,▁█▆▆▇▇▆▅
eval/steps_per_second,▁█▆▆▇▇▆▅
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▇▇▇▇▇▇▇███
train/grad_norm,▂▄▆▅▅▂▄▅▇▂▃█▄▂▂▃▃▃▂▅▃▄█▁▃▁▁▂▂▁▃▁▇▆▁▁▁▁▁▂


### Adapters model with BERT

Адаптер добавляется к слоям предобученной модели BERT, обеспечивая эффективную тонкую настройку с минимальным количеством обучаемых параметров.

В этом коде мы добавляем адаптер SeqBn с конфигурацией SeqBnConfig() к предобученной модели (например, BERT). Адаптеры позволяют эффективно дообучать большие языковые модели, замораживая основные слои и обучая только небольшое количество дополнительных параметров.

Ключевые особенности:

SeqBn — адаптер с последовательной пакетной нормализацией, улучшающей стабильность обучения.

Метод add_adapter() интегрирует адаптер в модель, а train_adapter() активирует его обучение.


https://docs.adapterhub.ml/methods.html#bottleneck-adapters

In [47]:
# Adapter configurations
adapter_name = "SeqBn"
adapter_config = SeqBnConfig()

Загружается bert-base-uncased - английская версия BERT, игнорирующая регистр букв.

Функция init() инициализирует адаптеры (добавляет SeqBn адаптер, как обсуждалось ранее)

In [48]:
model_adapt = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
init(model_adapt)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**add_adapter():**

Добавляет в модель адаптер с указанным именем (adapter_name)

Использует конфигурацию SeqBnConfig для инициализации

Сохраняет оригинальные параметры модели нетронутыми

**train_adapter():**

Переводит модель в режим обучения только адаптера

"Замораживает" все остальные параметры BERT

Значительно сокращает количество обучаемых параметров

**set_active_adapters():**

Активирует добавленный адаптер для последующего использования

Позволяет работать с разными адаптерами при необходимости

In [49]:
model_adapt.add_adapter(adapter_name, config=adapter_config)
model_adapt.train_adapter(adapter_name)
model_adapt.set_active_adapters(adapter_name)

In [50]:
training_args = TrainingArguments(
    output_dir="./bert_agnews_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch", #современный оптимизатор для трансформеров
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1", #выбор модели по F1-score
    greater_is_better=True,
    report_to="wandb", # визуализация в Weights & Biases
    logging_steps=50, #частота вывода логов
    save_total_limit=2, #предотвращает переполнение диска
    fp16=True,  # Автоматически использует mixed precision если доступно
    gradient_accumulation_steps=1, #полезно для больших батчей
    warmup_ratio=0.1, #плавное увеличение learning rate в начале
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
# Инициализация тренера для обучения модели
trainer  = Trainer(
    # Основные компоненты
    model= model_adapt,  # Загруженная ранее модель DistilBERT для классификации
    args=training_args,  # Параметры обучения, которые мы настроили ранее

    # Данные для обучения и оценки
    train_dataset=train_dataset,  # Обучающий датасет
    eval_dataset=eval_dataset,    # Валидационный датасет
    # Настройки обработки данных
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,  # Токенизатор для динамического добавления padding
        padding='longest',    # Дополнение до длины самого длинного примера в батче
        max_length=512,       # Максимальная длина последовательности (ограничение BERT)
        return_tensors='pt'   # Возвращать тензоры PyTorch
    ),

    # Метрики оценки
    compute_metrics=compute_metrics,  # Функция для расчета метрик качества

    # Коллбэки
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Остановить обучение, если метрика не улучшается 3 эпохи
            early_stopping_threshold=0.01  # Минимальное улучшение для продолжения обучения
        )
    ]
)


In [52]:
# Инициализация мониторинга в Weights & Biases
wandb.init(
    project="SentAnalysis_BERT_vs_Adapter_DistilBERT",  # Название проекта в W&B
    config=training_args.to_dict(),  # Сохранение всех параметров обучения
    name=f"BERT-base_Adapter-{datetime.now().strftime('%Y-%m-%d_%H-%M')}",  # Уникальное имя запуска
    tags=["baseline", "bert-base"]  # Теги для фильтрации экспериментов
)

try:
    # Основной цикл обучения
    trainer.train()  # Запуск обучения с ранней остановкой
    trainer.evaluate()
finally:
    # Обязательное завершение сессии W&B
    wandb.finish()

    # Оптимизация использования памяти GPU
    torch.cuda.empty_cache()  # Очистка кэша CUDA
    gc.collect()             # Запуск сборщика мусора


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.635500,0.527199,0.825625,0.825625,0.831960,0.822083
2,0.318600,0.303489,0.901875,0.901875,0.902312,0.901664


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.635500,0.527199,0.825625,0.825625,0.831960,0.822083
2,0.318600,0.303489,0.901875,0.901875,0.902312,0.901664
3,0.295100,0.280157,0.907917,0.907917,0.907949,0.907689
4,0.279600,0.266072,0.912708,0.912708,0.912736,0.912594
5,0.287900,0.257914,0.915312,0.915312,0.915294,0.915159


There were missing keys in the checkpoint model loaded: ['bert.prompt_tuning.base_model_embeddings.weight'].


eval/accuracy,▁▇▇███
eval/f1,▁▇▇███
eval/loss,█▂▂▁▁▁
eval/precision,▁▇▇███
eval/recall,▁▇▇███
eval/runtime,▂▆▂▃▁█
eval/samples_per_second,▇▂▇▆█▁
eval/steps_per_second,▇▂▇▆█▁
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▇█▃▄▄▇▆▄▃▅▄▄▅▄▆▁▃▂▃▃▃▅▄▃▃▂▅▃▃▂▆▇▂█▃▃▄▆▄▃


### Adapters model with DistilBERT

Аналогично добавим адаптер к DistilBERT
https://docs.adapterhub.ml/methods.html#bottleneck-adapters

In [64]:
# Adapter configurations
adapter_name = "SeqBn"
adapter_config = SeqBnConfig()

In [65]:
model_adapt = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)
init(model_adapt)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
model_adapt.add_adapter(adapter_name, config=adapter_config)
model_adapt.train_adapter(adapter_name)
model_adapt.set_active_adapters(adapter_name)

In [67]:
training_args = TrainingArguments(
    output_dir="./bert_agnews_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim="adamw_torch", #современный оптимизатор для трансформеров
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1", #выбор модели по F1-score
    greater_is_better=True,
    report_to="wandb", # визуализация в Weights & Biases
    logging_steps=50, #частота вывода логов
    save_total_limit=2, #предотвращает переполнение диска
    fp16=True,  # Автоматически использует mixed precision если доступно
    gradient_accumulation_steps=1, #полезно для больших батчей
    warmup_ratio=0.1, #плавное увеличение learning rate в начале
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [68]:
# Инициализация тренера для обучения модели
trainer  = Trainer(
    # Основные компоненты
    model= model_adapt,  # Загруженная ранее модель DistilBERT для классификации
    args=training_args,  # Параметры обучения, которые мы настроили ранее

    # Данные для обучения и оценки
    train_dataset=train_dataset,  # Обучающий датасет
    eval_dataset=eval_dataset,    # Валидационный датасет
    # Настройки обработки данных
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,  # Токенизатор для динамического добавления padding
        padding='longest',    # Дополнение до длины самого длинного примера в батче
        max_length=512,       # Максимальная длина последовательности (ограничение BERT)
        return_tensors='pt'   # Возвращать тензоры PyTorch
    ),

    # Метрики оценки
    compute_metrics=compute_metrics,  # Функция для расчета метрик качества

    # Коллбэки
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Остановить обучение, если метрика не улучшается 3 эпохи
            early_stopping_threshold=0.01  # Минимальное улучшение для продолжения обучения
        )
    ]
)


In [69]:
# Инициализация мониторинга в Weights & Biases
wandb.init(
    project="SentAnalysis_BERT_vs_Adapter_DistilBERT",  # Название проекта в W&B
    config=training_args.to_dict(),  # Сохранение всех параметров обучения
    name=f"DistilBERT-base_Adapter-{datetime.now().strftime('%Y-%m-%d_%H-%M')}",  # Уникальное имя запуска
    tags=["baseline", "bert-base"]  # Теги для фильтрации экспериментов
)

try:
    # Основной цикл обучения
    trainer.train()  # Запуск обучения с ранней остановкой
    trainer.evaluate()
finally:
    # Обязательное завершение сессии W&B
    wandb.finish()

    # Оптимизация использования памяти GPU
    torch.cuda.empty_cache()  # Очистка кэша CUDA
    gc.collect()             # Запуск сборщика мусора


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.346600,0.314326,0.889792,0.889792,0.889847,0.889663
2,0.301100,0.282815,0.900417,0.900417,0.901130,0.899974


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.346600,0.314326,0.889792,0.889792,0.889847,0.889663
2,0.301100,0.282815,0.900417,0.900417,0.901130,0.899974
3,0.268000,0.264787,0.905729,0.905729,0.905909,0.905457
4,0.258800,0.255131,0.908750,0.908750,0.908724,0.908488
5,0.278900,0.249614,0.910833,0.910833,0.911230,0.910674


There were missing keys in the checkpoint model loaded: ['distilbert.prompt_tuning.base_model_embeddings.weight'].


eval/accuracy,▁▅▆▇██
eval/f1,▁▄▆▇██
eval/loss,█▅▃▂▁▁
eval/precision,▁▅▆▇██
eval/recall,▁▅▆▇██
eval/runtime,▃▃▁▄▄█
eval/samples_per_second,▆▅█▅▅▁
eval/steps_per_second,▆▅█▅▅▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▁▂▂▂▃▂▃▁▁▄▂▃▃▃▂▃▂▂▅▁▃▁█▂▃▄▂▂▄▂▁▂▂▃▅▂▂▄▁▃
